In [1]:
# imports
import os
import glob
import numpy as np
import pandas as pd 
import altair as alt
# import altair_viewer

In [2]:
# load in dataframes and concatenate together
path = (
    "C:/Users/jacob.hester/Documents/Python_scripts/"\
    "Historical_beef_cattle_prices/data/processed"
    )
raw_files_path = os.path.join(path, "*.csv")
df_list = []
for f in glob.glob(raw_files_path):
    print(f)
    df = pd.read_csv(f, index_col=None, header=0)
    df_list.append(df)
df = pd.concat(df_list, ignore_index = True)

C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2004_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2005_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2006_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2007_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2008_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2009_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2011_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2012_prices.csv
C:/Users/jacob.hester/Documents/Python_scripts/Historical_beef_cattle_prices/data/processed\2013_prices.csv
C:/Users/jacob.hester/Docume

In [3]:
# check data types
df.dtypes

Year              int64
Type             object
Weight_group     object
Hd Cnt           object
Jan             float64
Feb             float64
Mar             float64
Apr             float64
May             float64
Jun             float64
Jul             float64
Aug             float64
Sep             float64
Oct             float64
Nov             float64
Dec             float64
Avg             float64
dtype: object

In [4]:
# rename Hd Cnt col and change type 
# NOTE: column was read in as object type because of commas in head counts
df.rename(columns = {'Hd Cnt' : 'Hd_cnt'}, inplace = True)
df['Hd_cnt'] = df['Hd_cnt'].str.replace(',', '')
data_types_dictionary = {'Hd_cnt' : float}
df = df.astype(data_types_dictionary)

In [5]:
# check for missing values 
df.isnull().sum()

Year               0
Type               0
Weight_group       0
Hd_cnt             0
Jan             1546
Feb             1527
Mar             1468
Apr             1444
May             1533
Jun             1533
Jul             1685
Aug             1544
Sep             1557
Oct             1459
Nov             1553
Dec             1683
Avg                0
dtype: int64

In [6]:
# look at overall sales per year 
sales_per_yr_df = df.groupby(['Year'])['Hd_cnt'].sum()
sales_per_yr_df = pd.DataFrame(sales_per_yr_df)
sales_per_yr_df.reset_index(inplace = True)

# change Year to datetime type 
sales_per_yr_df['Year'] = pd.to_datetime(
    sales_per_yr_df['Year'].astype(str), format='%Y'
    )

In [7]:
# sales per year visual
# alt.renderers.enable('altair_viewer')

# add x index values for visual
x_year_domain = ['2003-01-01', '2019-01-01']
sales_per_year = alt.Chart(sales_per_yr_df).mark_bar(size = 20).encode(
    x = alt.X(
        'Year', 
        axis = alt.Axis(
            titleFontSize = 14
            ),
        scale = alt.Scale(domain = x_year_domain)
            ),
    y = alt.Y(
        'Hd_cnt', 
        axis = alt.Axis(
            title = 'Head Count',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Overall Sales Per Year",
                "subtitle" : "Beef Cattle Sales in Alabama"
                },
            width = 700
    ).configure_axisY(
        titleAngle = 0,
        titleX = -100,
    ).configure_axisX(
        titleX = 350,
        titleY = 30
    ).configure_title(
        fontSize = 18   
    )   
sales_per_year

alt.Chart(...)

In [8]:
# look at average overall prices per year
prices_per_yr_df = df.groupby(['Year'])['Avg'].mean()
prices_per_yr_df = pd.DataFrame(prices_per_yr_df)
prices_per_yr_df.reset_index(inplace = True)
prices_per_yr_df['Year'] = pd.to_datetime(
    prices_per_yr_df['Year'].astype(str), format='%Y'
    )
prices_per_yr_df.head()

prices_per_year = alt.Chart(prices_per_yr_df).mark_bar(size = 20).encode(
    x = alt.X(
        'Year', 
        axis = alt.Axis(
            titleFontSize = 14
            ),
        scale = alt.Scale(domain = x_year_domain)
            ),
    y = alt.Y(
        'Avg', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Price Per Year",
                "subtitle" : "Beef Cattle Sales in Alabama"
                },
            width = 700
    ).configure_axisY(
        titleAngle = 0,
        titleX = -100,
    ).configure_axisX(
        titleX = 350,
        titleY = 30
    ).configure_title(
        fontSize = 18   
    )
prices_per_year

alt.Chart(...)

In [9]:
# look at average prices per month 
price_per_month_df = pd.melt(
    df, 
    id_vars = ['Year'], 
    value_vars = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)
# start by dropping missing values 
price_per_month_df = price_per_month_df.dropna()

price_per_month_df = price_per_month_df.groupby(['Year', 'variable'])['value'].mean()
price_per_month_df = pd.DataFrame(price_per_month_df)
price_per_month_df.reset_index(inplace = True)
price_per_month_df['Year'] = pd.to_datetime(
    price_per_month_df['Year'].astype(str), format='%Y'
)

In [10]:
# add column for month and year 
months_dict = {'Jan' : 1, 
               'Feb' : 2,
               'Mar' : 3, 
               'Apr' : 4, 
               'May' : 5, 
               'Jun' : 6, 
               'Jul' : 7, 
               'Aug' : 8, 
               'Sep' : 9, 
               'Oct' : 10, 
               'Nov' : 11, 
               'Dec' : 12
              }
price_per_month_df['month'] = price_per_month_df['variable'].map(months_dict)
price_per_month_df['year'] = price_per_month_df['Year'].dt.year

In [11]:
price_per_month_df.head()

,Year,variable,value,month,year
0,2004-01-01,Apr,98.259673,4,2004
1,2004-01-01,Aug,111.039098,8,2004
2,2004-01-01,Dec,105.845301,12,2004
3,2004-01-01,Feb,176.858529,2,2004
4,2004-01-01,Jan,125.400099,1,2004


In [12]:
price_per_month_df.dtypes

Year        datetime64[ns]
variable            object
value              float64
month                int64
year                 int64
dtype: object

In [13]:
price_per_month_df['day'] = 1
price_per_month_df.head()

,Year,variable,value,month,year,day
0,2004-01-01,Apr,98.259673,4,2004,1
1,2004-01-01,Aug,111.039098,8,2004,1
2,2004-01-01,Dec,105.845301,12,2004,1
3,2004-01-01,Feb,176.858529,2,2004,1
4,2004-01-01,Jan,125.400099,1,2004,1


In [14]:
price_per_month_df['date_col'] = pd.to_datetime(price_per_month_df[['month', 'year', 'day']])

In [15]:
print(price_per_month_df.head(25))
print('')
print(price_per_month_df.dtypes)

         Year variable       value  month  year  day   date_col
0  2004-01-01      Apr   98.259673      4  2004    1 2004-04-01
1  2004-01-01      Aug  111.039098      8  2004    1 2004-08-01
2  2004-01-01      Dec  105.845301     12  2004    1 2004-12-01
3  2004-01-01      Feb  176.858529      2  2004    1 2004-02-01
4  2004-01-01      Jan  125.400099      1  2004    1 2004-01-01
5  2004-01-01      Jul  110.345911      7  2004    1 2004-07-01
6  2004-01-01      Jun  106.885388      6  2004    1 2004-06-01
7  2004-01-01      Mar   96.656243      3  2004    1 2004-03-01
8  2004-01-01      May  103.132754      5  2004    1 2004-05-01
9  2004-01-01      Nov  105.206058     11  2004    1 2004-11-01
10 2004-01-01      Oct  105.829586     10  2004    1 2004-10-01
11 2004-01-01      Sep  107.097124      9  2004    1 2004-09-01
12 2005-01-01      Apr  116.129494      4  2005    1 2005-04-01
13 2005-01-01      Aug  107.362072      8  2005    1 2005-08-01
14 2005-01-01      Dec  111.994426     1

In [16]:
# prices per month over time visual
prices_per_month = alt.Chart(price_per_month_df).mark_point().encode(
    x = alt.X(
        'date_col', 
        axis = alt.Axis(
            title = 'Date',
            titleFontSize = 14
            ),
            ),
    y = alt.Y(
        'value', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Price Per Month",
                "subtitle" : "Average Monthly Beef Cattle Price in Alabama, 2004-2018"
                },
            width = 2000
    ).configure_axisY(
        titleAngle = 0,
        titleX = -100,
    ).configure_axisX(
        titleX = 1000,
        titleY = 30
    ).configure_title(
        fontSize = 18   
    )
prices_per_month

alt.Chart(...)

In [17]:
# look at what month is highest each year 
max_vals_dict = price_per_month_df.groupby(['year'])['value'].max().to_dict()
print(max_vals_dict)

{2004: 176.8585294117647, 2005: 116.51857723577231, 2006: 115.4780073800739, 2007: 105.3419305019305, 2008: 97.15622580645166, 2009: 89.99515789473686, 2011: 127.42837837837835, 2012: 155.7026724137931, 2013: 152.78152, 2014: 247.43409836065572, 2015: 246.97152173913048, 2016: 162.55872000000002, 2017: 136.1089705882353, 2018: 141.3530935251799}


In [18]:
price_per_month_df['max_price'] = price_per_month_df['year'].map(max_vals_dict)
print(price_per_month_df.head())

        Year variable       value  month  year  day   date_col   max_price
0 2004-01-01      Apr   98.259673      4  2004    1 2004-04-01  176.858529
1 2004-01-01      Aug  111.039098      8  2004    1 2004-08-01  176.858529
2 2004-01-01      Dec  105.845301     12  2004    1 2004-12-01  176.858529
3 2004-01-01      Feb  176.858529      2  2004    1 2004-02-01  176.858529
4 2004-01-01      Jan  125.400099      1  2004    1 2004-01-01  176.858529


In [19]:
price_per_month_df['max_price_month'] = np.where(price_per_month_df['value'] == price_per_month_df['max_price'], 1, 0)
print(price_per_month_df.head())

        Year variable       value  month  year  day   date_col   max_price  \
0 2004-01-01      Apr   98.259673      4  2004    1 2004-04-01  176.858529   
1 2004-01-01      Aug  111.039098      8  2004    1 2004-08-01  176.858529   
2 2004-01-01      Dec  105.845301     12  2004    1 2004-12-01  176.858529   
3 2004-01-01      Feb  176.858529      2  2004    1 2004-02-01  176.858529   
4 2004-01-01      Jan  125.400099      1  2004    1 2004-01-01  176.858529   

   max_price_month  
0                0  
1                0  
2                0  
3                1  
4                0  


In [20]:
prices_per_month = alt.Chart(price_per_month_df).mark_point().encode(
     color = 'max_price_month',
    x = alt.X(
        'date_col', 
        axis = alt.Axis(
            title = 'Date',
            titleFontSize = 14
            ),
            ),
    y = alt.Y(
        'value', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Price Per Month",
                "subtitle" : "Average Monthly Beef Cattle Price in Alabama, 2004-2018"
                },
            width = 2000
    )

text = (
    alt.Chart(price_per_month_df.query("max_price_month == 1"))
    .mark_text(dy=-15, color="darkblue")
    .encode(x=alt.X("date_col"), y=alt.Y("value"), text=alt.Text("variable"))
)

(prices_per_month + text)

alt.LayerChart(...)

In [21]:
# look at feeder steers 
steers_df = df[df['Type'].str.contains("Feeder Steers")]
print(steers_df.head())

    Year                    Type Weight_group  Hd_cnt  Jan  Feb  Mar     Apr  \
15  2004  Feeder Steers: Large 1  350-400 DBD     1.0  NaN  NaN  NaN  126.00   
16  2004  Feeder Steers: Large 1  450-500 DBF     1.0  NaN  NaN  NaN  113.00   
17  2004  Feeder Steers: Large 1  500-550 DBG     1.0  NaN  NaN  NaN  106.50   
18  2004  Feeder Steers: Large 1  550-600 DBH     4.0  NaN  NaN  NaN  100.96   
19  2004  Feeder Steers: Large 1  600-650 DBI     4.0  NaN  NaN  NaN     NaN   

    May    Jun  Jul     Aug    Sep    Oct  Nov  Dec     Avg  
15  NaN    NaN  NaN     NaN    NaN    NaN  NaN  NaN  126.00  
16  NaN    NaN  NaN     NaN    NaN    NaN  NaN  NaN  113.00  
17  NaN    NaN  NaN     NaN    NaN    NaN  NaN  NaN  106.50  
18  NaN  102.0  NaN     NaN    NaN    NaN  NaN  NaN  101.21  
19  NaN    NaN  NaN  116.51  105.0  102.0  NaN  NaN  109.87  


In [22]:
steers_df = pd.melt(
    steers_df, 
    id_vars = ['Year'], 
    value_vars = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)
print(steers_df.head())

   Year variable  value
0  2004      Jan    NaN
1  2004      Jan    NaN
2  2004      Jan    NaN
3  2004      Jan    NaN
4  2004      Jan    NaN


In [23]:
# start by dropping missing values 
steers_df = steers_df.dropna()
steers_df = steers_df.groupby(['Year', 'variable'])['value'].mean()
print(steers_df.head())

Year  variable
2004  Apr         106.918657
      Aug         119.204868
      Dec         114.327733
      Feb         104.237727
      Jan         102.081818
Name: value, dtype: float64


In [24]:
steers_df = pd.DataFrame(steers_df)
steers_df.reset_index(inplace = True)
steers_df['Year'] = pd.to_datetime(
    steers_df['Year'].astype(str), format='%Y'
)

In [25]:
print(steers_df.head())
print(' ')
print(steers_df.dtypes)

        Year variable       value
0 2004-01-01      Apr  106.918657
1 2004-01-01      Aug  119.204868
2 2004-01-01      Dec  114.327733
3 2004-01-01      Feb  104.237727
4 2004-01-01      Jan  102.081818
 
Year        datetime64[ns]
variable            object
value              float64
dtype: object


In [26]:
steers_df['month'] = steers_df['variable'].map(months_dict)
steers_df['year'] = steers_df['Year'].dt.year

In [27]:
steers_df['day'] = 1
steers_df['date_col'] = pd.to_datetime(steers_df[['month', 'year', 'day']])
max_vals_dict = steers_df.groupby(['year'])['value'].max().to_dict()
steers_df['max_price'] = steers_df['year'].map(max_vals_dict)
steers_df['max_price_month'] = np.where(steers_df['value'] == steers_df['max_price'], 1, 0)

In [28]:
steer_prices_per_month = alt.Chart(steers_df).mark_point().encode(
     color = 'max_price_month',
    x = alt.X(
        'date_col', 
        axis = alt.Axis(
            title = 'Date',
            titleFontSize = 14
            ),
            ),
    y = alt.Y(
        'value', 
        axis = alt.Axis(
            title = 'Average Price',
            titleFontSize = 14
            )
        )
    ).properties(
            title = {
                "text" : "Average Feeder Steer Price Per Month",
                "subtitle" : "Average Monthly Beef Cattle Price in Alabama, 2004-2018"
                },
            width = 2000
    )

steer_text = (
    alt.Chart(steers_df.query("max_price_month == 1"))
    .mark_text(dy=-15, color="darkblue")
    .encode(x=alt.X("date_col"), y=alt.Y("value"), text=alt.Text("variable"))
)

(steer_prices_per_month + steer_text)

alt.LayerChart(...)

In [29]:
# what is the average month highest price? 
steers_df.head()

,Year,variable,value,month,year,day,date_col,max_price,max_price_month
0,2004-01-01,Apr,106.918657,4,2004,1,2004-04-01,120.410508,0
1,2004-01-01,Aug,119.204868,8,2004,1,2004-08-01,120.410508,0
2,2004-01-01,Dec,114.327733,12,2004,1,2004-12-01,120.410508,0
3,2004-01-01,Feb,104.237727,2,2004,1,2004-02-01,120.410508,0
4,2004-01-01,Jan,102.081818,1,2004,1,2004-01-01,120.410508,0


In [30]:
# filter steers_df to max_price_month = 1 and take the average of 
steers_df_max = steers_df[steers_df['max_price_month'] == 1]
steers_df_max.head(20)

,Year,variable,value,month,year,day,date_col,max_price,max_price_month
5,2004-01-01,Jul,120.410508,7,2004,1,2004-07-01,120.410508,1
12,2005-01-01,Apr,124.540132,4,2005,1,2005-04-01,124.540132,1
27,2006-01-01,Feb,123.006184,2,2006,1,2006-02-01,123.006184,1
37,2007-01-01,Aug,114.869130,8,2007,1,2007-08-01,114.869130,1
51,2008-01-01,Feb,104.992674,2,2008,1,2008-02-01,104.992674,1
68,2009-01-01,May,97.196154,5,2009,1,2009-05-01,97.196154,1
74,2011-01-01,Dec,138.020204,12,2011,1,2011-12-01,138.020204,1
87,2012-01-01,Feb,172.061389,2,2012,1,2012-02-01,172.061389,1
98,2013-01-01,Dec,170.718529,12,2013,1,2013-12-01,170.718529,1
110,2014-01-01,Dec,276.456571,12,2014,1,2014-12-01,276.456571,1


In [31]:
# filter out december months 
steers_df_max = steers_df_max[steers_df['month'] != 12]
steers_df_max.head(20)

<ipython-input-31-2c9f4e0252d4>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  steers_df_max = steers_df_max[steers_df['month'] != 12]


,Year,variable,value,month,year,day,date_col,max_price,max_price_month
5,2004-01-01,Jul,120.410508,7,2004,1,2004-07-01,120.410508,1
12,2005-01-01,Apr,124.540132,4,2005,1,2005-04-01,124.540132,1
27,2006-01-01,Feb,123.006184,2,2006,1,2006-02-01,123.006184,1
37,2007-01-01,Aug,114.869130,8,2007,1,2007-08-01,114.869130,1
51,2008-01-01,Feb,104.992674,2,2008,1,2008-02-01,104.992674,1
68,2009-01-01,May,97.196154,5,2009,1,2009-05-01,97.196154,1
87,2012-01-01,Feb,172.061389,2,2012,1,2012-02-01,172.061389,1
127,2015-01-01,Mar,275.865714,3,2015,1,2015-03-01,275.865714,1
139,2016-01-01,Mar,178.248919,3,2016,1,2016-03-01,178.248919,1
152,2017-01-01,May,151.692750,5,2017,1,2017-05-01,151.692750,1


In [32]:
avg_month_val  = steers_df_max['month'].mean()
print(avg_month_val)

4.0


In [33]:
# look at all levels in Type column
for type in sorted(df.Type.unique()):
    print(type)

Cow Calf Pairs: Large 2 Aged
Cow Calf Pairs: Large 2 Middle Aged
Cow Calf Pairs: Medium 1-2 Aged
Cow Calf Pairs: Medium 1-2 Middle Aged
Cow Calf Pairs: Medium 2 Aged
Cow Calf Pairs: Medium 2 Middle Aged
Cow Calf Pairs: Medium 3 Middle Aged
Cow Calf Pairs: Medium and Large 2 Middle Aged
Cow Calf Pairs: Small 1 Young
Cow Calf Pairs: Small 1-2 Aged
Cow Calf Pairs: Small 1-2 Middle Aged
Cow Calf Pairs: Small 2 Aged
Cow Calf Pairs: Small 2 Middle Aged
Feeder Bulls: Large 1
Feeder Bulls: Large 2
Feeder Bulls: Large 3
Feeder Bulls: Medium 1
Feeder Bulls: Medium 2
Feeder Bulls: Medium 3
Feeder Bulls: Medium and Large 1
Feeder Bulls: Medium and Large 1-2
Feeder Bulls: Medium and Large 2
Feeder Bulls: Medium and Large 2-3
Feeder Bulls: Medium and Large 3
Feeder Bulls: Small 1
Feeder Bulls: Small 2
Feeder Bulls: Small 2-3
Feeder Bulls: Small 3
Feeder Bulls: Small and Medium 1
Feeder Bulls: Small and Medium 1-2
Feeder Bulls: Small and Medium 2
Feeder Bulls: Small and Medium 2-3
Feeder Bulls: Small

In [34]:
# split column on colon
df[['Overall_type', 'Sub_type']] = df['Type'].str.split(':', 1, expand=True)
print(df.head())

   Year                            Type   Weight_group   Hd_cnt    Jan    Feb  \
0  2004   Slaughter Heifers: Select 2-3  1200-1400 JAJ     22.0    NaN    NaN   
1  2004  Slaughter Cows: Breakers 75-80   850-1200 BAA   2759.0    NaN  46.34   
2  2004  Slaughter Cows: Breakers 75-80  1200-1600 BAH   5636.0  44.57  47.64   
3  2004  Slaughter Cows: Breakers 75-80  1600-2000 BAK    328.0    NaN    NaN   
4  2004    Slaughter Cows: Boners 80-85   800-1200 BAB  14218.0  43.60  43.62   

     Mar    Apr    May    Jun    Jul    Aug    Sep    Oct    Nov    Dec  \
0    NaN    NaN  69.23    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1  45.35  47.95  50.96  52.74  54.33  52.46  50.17  47.41  48.24  47.47   
2  46.91  49.12  53.03  55.07  55.86  53.74  52.06  48.57  49.93  48.98   
3  48.97  48.30  55.40  56.68  57.25  54.63  52.89  48.88  50.32  49.24   
4  45.71  47.23  51.25  52.54  54.01  52.66  52.13  47.84  49.31  48.67   

     Avg       Overall_type         Sub_type  
0  69.23  Slaug

In [35]:
for type in sorted(df.Overall_type.unique()):
    print(type)

Cow Calf Pairs
Feeder Bulls
Feeder Heifers
Feeder Holstein Steers
Feeder Steers
Slaughter Bulls
Slaughter Cows
Slaughter Heifers
Stock Cows


In [36]:
# look at price series for Feeder Bulls, Heifers, and Steers 
feeder_df = df[df['Overall_type'].isin(['Feeder Bulls', 'Feeder Heifers', 'Feeder Steers'])]

In [37]:
feeder_df.head() 

,Year,Type,Weight_group,Hd_cnt,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg,Overall_type,Sub_type
15,2004,Feeder Steers: Large 1,350-400 DBD,1.0,NaN,NaN,NaN,126.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.00,Feeder Steers,Large 1
16,2004,Feeder Steers: Large 1,450-500 DBF,1.0,NaN,NaN,NaN,113.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.00,Feeder Steers,Large 1
17,2004,Feeder Steers: Large 1,500-550 DBG,1.0,NaN,NaN,NaN,106.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.50,Feeder Steers,Large 1
18,2004,Feeder Steers: Large 1,550-600 DBH,4.0,NaN,NaN,NaN,100.96,NaN,102.0,NaN,NaN,NaN,NaN,NaN,NaN,101.21,Feeder Steers,Large 1
19,2004,Feeder Steers: Large 1,600-650 DBI,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.51,105.0,102.0,NaN,NaN,109.87,Feeder Steers,Large 1
